Business Intelligence

Time Series Analysis
- Are there monthly and weekly sales seasonality?
- Are there anomalous* peaks and troughs - what could explain them?
- How does every year from 2015-2019 compare against each other?
  
Distribution and Aggregations
- What's the distribution of sales per product category^? 
- What's the dollar value per month/year per:
  - individual
  - cluster (once identified)
  - product category

In [1]:
import pandas as pd
import numpy as np

In [29]:
# member = pd.read_csv("member_logs.csv")
# guest = pd.read_csv("cl_guest.csv")

In [21]:
df = member[["acn","timestamp","item","item_group","total","sys_id"]]
df = df.rename(columns={"acn":"id","sys_id":"product_category"})

In [33]:
df2 = guest[["sys_id","timestamp","item","item_group","total"]]
df2 = df2.rename(columns={"sys_id":"id"})
df2["product_category"] = "Guest"

In [73]:
len(df)+len(df2)

143718

**Task: Concat df and df2**  
But first, check if ids are unique, if not find out why not.

In [46]:
member.head()

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
0,0,7315,2015-01-03 11:00:00,Pia Lomax,Lessons with Joe Snailum (01/03/2015),Lessons,Paid,1.0,65.00,0.00,0.00,65.00,65.00,0.0,0.0,79,General Single Membership
1,1,37831,2015-08-12 21:30:00,Jennifer Hernandez,$1 Snacks,Snacks,Paid,1.0,0.92,0.00,0.08,1.00,1.00,0.0,0.0,566,General Family Membership
2,2,37844,2015-08-13 10:50:00,Douglas Draeger,Java Jolt,Smoothie,Paid,1.0,5.31,0.53,0.39,5.17,5.17,0.0,0.0,1532,General Property Owner
3,3,37843,2015-08-13 10:50:00,Douglas Draeger,Java Jolt,Smoothie,Paid,1.0,5.31,0.53,0.39,5.17,5.17,0.0,0.0,1532,General Property Owner
4,4,37842,2015-08-13 10:50:00,Douglas Draeger,Mango Tango,Smoothie,Paid,1.0,4.39,0.44,0.33,4.28,4.28,0.0,0.0,1532,General Property Owner


In [72]:
df[df.id.isin(df2.id)].id.count()

971

In [48]:
dups = df[df.id.isin(df2.id)].id.unique()

In [50]:
member[member.acn.isin(dups)].name.unique()

array(['Ashley Lloyd', 'Lee Stacy', 'Rhonda Stacy', 'Diego Garcia',
       'Hartley Clark', 'Evyn Clark', 'Abigail Kelly', 'Cole Jecmenek',
       'David Smith', 'Khloe Villa', 'Aaron Malakoff', 'Jessica Villa',
       'Ryan Villa'], dtype=object)

Check from the cancellation data.

In [70]:
# cancel = pd.read_csv("cancellation.csv")
cancel[cancel["Last name"]=="Malakoff"]

,User Group Name,First name,Last name,Access Card Number,Member Since,Profile Creation Date,Street address,System ID,ZIP code,Cancellation Date,Cancellation Reason,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
247,General Single Membership,Aaron,Malakoff,NaN,1/20/2018,1/20/2018,127 E Edgewater Terrace,3123,78130.0,10/23/2019,Non-Usage,NaN,NaN,NaN,NaN


Duplicated entries who are **confirmed** in the Cancellation data

- Aaron Malakoff  
- Rhonda, Lee Stacy  
- David Smith

Duplicated entries who have **similar names** in the Cancellation data
- Roberto same as Diego Garcia?  
- Wes same as Hartley/Evyn Clark?  
- Jarrett same as Cole Jecmenek?

Duplicated entries who are **not** in the Cancellation data
- Ashley Lloyd  
- Abigail Kelly
- Jessica Villa

In [78]:
member.item_group.unique().tolist()

['Lessons',
 'Snacks',
 'Smoothie',
 'White Wine',
 'Drinks',
 'Tips',
 'Guest Fee',
 'Apparel',
 'Accessories',
 'Tennis Balls',
 'Adult Clinics',
 'Massage',
 'Membership Initiation ',
 'Membership Dues',
 'Junior Clinics',
 'Beer',
 'Date Night',
 'Racquet Rental',
 'Red Wine',
 'String',
 'Package Liability',
 'Restring/ Labor Only',
 'Special Event',
 'Shoes',
 'Facility Rental',
 'Dinners',
 'Camp',
 'Racquets',
 'Leagues',
 'Personal Training',
 'Catie Johnson Home School',
 'Fitness Special Event',
 'Pro Shop',
 'Gift Certificate Liability',
 'Social Event',
 'Swim Lessons ',
 'Credits',
 'Swim Camp',
 'TRAIN4LIFE',
 'Mini Bees Soccer',
 'Teen Performance Camp',
 'CDM- MISC',
 'Soccer- MSOS',
 'Special Event-Non Taxable (Includes Instruction)',
 'Food Allowance',
 'Naturally Cafe',
 'Naturally-Salad',
 'Naturally Lunch Special',
 'Naturally Juices',
 'Naturally-Smoothies',
 'Naturally- Sandwiches',
 'Court Time',
 'VIP TENNIS',
 'VIP FITNESS',
 'Food & Beverage',
 'Kid Fit',
 '

In [80]:
member[member.item_group=="Food Allowance"]

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
68152,125,173879,2018-08-31 00:00:00,Nancy Aubin,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,230,General Family Membership
68153,126,173878,2018-08-31 00:00:00,Matt Ashabranner,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,2552,General Property Owner
68154,127,173878,2018-08-31 00:00:00,Matt Ashabranner,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,2552,Premier Family Membership
68155,128,173878,2018-08-31 00:00:00,Matt Ashabranner,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,2552,General Family Membership
68156,129,173877,2018-08-31 00:00:00,Scott Anderson,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,2195,General Family Membership
68157,130,173877,2018-08-31 00:00:00,Scott Anderson,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,2195,General Single Membership
68158,131,173876,2018-08-31 00:00:00,Natalia Alpatova,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,2642,General Family Membership
68159,132,173875,2018-08-31 00:00:00,Jim Albright,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,245,General Family Membership
68160,133,173874,2018-08-31 00:00:00,Warren Albrecht,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,236,General Family Membership
68161,134,173873,2018-08-31 00:00:00,Mark Adams,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,3183,General Family Membership


In [77]:
member.item.unique().tolist()

['Lessons with Joe Snailum (01/03/2015)',
 '$1 Snacks',
 'Java Jolt',
 'Mango Tango',
 'Peanut Butter Cup',
 'Nicolas Feullate Blue Brut',
 'Powerade',
 'Tips',
 'Jacobs Creek Chardonnay Glass',
 'Adult Guest Fee',
 'G SP Watercolor Tank',
 'G SP Watercolor 11 Skirt',
 'JNCC HAT',
 'Dasani Water bottle',
 'Smoothie Add-On',
 'Vitamin Water',
 '$1.75 Ice Cream',
 'Core Power',
 'Strawberry Classic',
 '$2.00 Ice Cream',
 'Tennis Balls',
 'CLIFF BAR',
 'SOCKS',
 'Group Event: Open Clinic (08/13/2015)',
 'Group Event: Instant Tennis (08/12/2015)',
 'Member 1 Hour Massage',
 'Membership Initiation',
 'Membership for &quot;General Single Membership&quot; (August 2015)',
 'Lessons with Jerel Pleasant (08/13/2015)',
 'Lessons with Joe Snailum (08/08/2015)',
 'Lessons with Jerel Pleasant (08/12/2015)',
 'Soda',
 'Program: Joeys | 1: Sept 7 - Oct 16 | Mondays 04:00pm - 04:30pm',
 'Michelob Ultra Bottle',
 'Shiner Bock',
 'Mumm Napa Brut Rose Champagne',
 'Stella Artois',
 'Program: Wallabies | 1